![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Spark NLP and Spark ML Pipelines

In [1]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.3

# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.2.2

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)
     |████████████████████████████████| 215.6MB 125kB/s 
     |████████████████████████████████| 204kB 39.6MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.3-py2.py3-none-any.whl size=215964963 sha256=bd5c6069dd9ab3f9cffc13cd7533b8a44ed40fe831e8042417c2b8327c7f714d
  Stored in directory: /root/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark
     |████████████████████████████████| 71kB 4.7MB/s 


## Simple Topic Modeling

`Spark-NLP`
* DocumentAssembler
* SentenceDetector
* Tokenizer
* Normalizer
* POS tagger
* Chunker
* Finisher

`Spark ML`
* Hashing
* TF-IDF
* LDA

In [0]:
import sys
import time

from pyspark.sql.functions import col
from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, Tokenizer
from pyspark.ml.clustering import LDA, LDAModel

#Spark NLP
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import *

### Let's create a Spark Session for our app

In [3]:
spark = sparknlp.start()

print("Spark NLP version")
sparknlp.version()
print("Apache Spark version")
spark.version

Spark NLP version
2.2.1
Apache Spark version


'2.4.3'

Let's download some scientific sample from PubMed dataset:
```
wget -N 	https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv -P /tmp
```

In [4]:
! wget -N 	https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv -P /tmp

--2019-09-20 16:56:07--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.139.157
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.139.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10484510 (10.0M) [text/csv]
Saving to: ‘/tmp/pubmed-sample.csv’

pubmed-sample.csv   100%[===================>]  10.00M  17.3MB/s    in 0.6s    

2019-09-20 16:56:08 (17.3 MB/s) - ‘/tmp/pubmed-sample.csv’ saved [10484510/10484510]



In [0]:
pubMedDF = spark.read\
                .option("header", "true")\
                .csv("/tmp/pubmed-sample.csv")\
                .filter("AB IS NOT null")\
                .withColumn("text", col("AB"))\
                .drop("TI", "AB")

In [6]:
pubMedDF.printSchema()
pubMedDF.show()

root
 |-- text: string (nullable = true)

+--------------------+
|                text|
+--------------------+
|The human KCNJ9 (...|
|BACKGROUND: At pr...|
|OBJECTIVE: To inv...|
|Combined EEG/fMRI...|
|Kohlschutter synd...|
|Statistical analy...|
|The synthetic DOX...|
|Our objective was...|
|We conducted a ph...|
|"Monomeric sarcos...|
|We presented the ...|
|The literature de...|
|A novel approach ...|
|An HPLC-ESI-MS-MS...|
|The localizing an...|
|OBJECTIVE: To eva...|
|For the construct...|
|We report the res...|
|Intraparenchymal ...|
|It is known that ...|
+--------------------+
only showing top 20 rows



In [7]:
pubMedDF.count()

7537

### Let's create Spark-NLP Pipeline

In [8]:
# Spark NLP Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

posTagger = PerceptronModel.pretrained() \
  .setInputCols(["sentence", "token"])

chunker = Chunker() \
    .setInputCols(["sentence", "pos"]) \
    .setOutputCol("chunk") \
    .setRegexParsers(["<NNP>+", "<DT>?<JJ>*<NN>"])

finisher = Finisher() \
  .setInputCols(["chunk"]) \
  .setIncludeMetadata(False)

nlpPipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector, 
    tokenizer,
    posTagger,
    chunker,
    finisher
])

pos_anc download started this may take some time.
Approximate size to download 4.3 MB
[OK!]


In [0]:
nlpPipelineDF = nlpPipeline.fit(pubMedDF).transform(pubMedDF)

### Let's create Spark ML Pipeline

In [0]:
# SPark ML Pipeline

cv = CountVectorizer(inputCol="finished_chunk", outputCol="features", vocabSize=1000, minDF=10.0, minTF=10.0)
idf = IDF(inputCol="features", outputCol="idf")
lda = LDA(k=10, maxIter=5)
### Let's create Spark-NLP Pipeline
mlPipeline = Pipeline(stages=[
    cv,
    idf,
    lda
])

### We are going to train Spark ML Pipeline by using Spark-NLP Pipeline

In [0]:
# Let's create Spark-NLP Pipeline
mlModel = mlPipeline.fit(nlpPipelineDF)

In [0]:
mlPipelineDF = mlModel.transform(nlpPipelineDF)

In [13]:
mlPipelineDF.show()

+--------------------+--------------------+------------+------------+--------------------+
|                text|      finished_chunk|    features|         idf|   topicDistribution|
+--------------------+--------------------+------------+------------+--------------------+
|The human KCNJ9 (...|[KCNJ9, Kir, GIRK...|(1000,[],[])|(1000,[],[])|[0.0,0.0,0.0,0.0,...|
|BACKGROUND: At pr...|[BACKGROUND, the ...|(1000,[],[])|(1000,[],[])|[0.0,0.0,0.0,0.0,...|
|OBJECTIVE: To inv...|[OBJECTIVE, =9796...|(1000,[],[])|(1000,[],[])|[0.0,0.0,0.0,0.0,...|
|Combined EEG/fMRI...|[Combined EEG/fMR...|(1000,[],[])|(1000,[],[])|[0.0,0.0,0.0,0.0,...|
|Kohlschutter synd...|[Kohlschutter, sy...|(1000,[],[])|(1000,[],[])|[0.0,0.0,0.0,0.0,...|
|Statistical analy...|[Statistical, ana...|(1000,[],[])|(1000,[],[])|[0.0,0.0,0.0,0.0,...|
|The synthetic DOX...|[DOX-LNA, conjuga...|(1000,[],[])|(1000,[],[])|[0.0,0.0,0.0,0.0,...|
|Our objective was...|[objective, blood...|(1000,[],[])|(1000,[],[])|[0.0,0.0,0.0,0.0,...|

In [0]:
ldaModel = mlModel.stages[2]

In [15]:
ll = ldaModel.logLikelihood(mlPipelineDF)
lp = ldaModel.logPerplexity(mlPipelineDF)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))


The lower bound on the log likelihood of the entire corpus: -19726.433220525225
The upper bound on perplexity: 12.884672253772191


In [16]:
# Describe topics.
print("The topics described by their top-weighted terms:")
ldaModel.describeTopics(3).show(truncate=False)

The topics described by their top-weighted terms:
+-----+---------------+--------------------------------------------------------------------+
|topic|termIndices    |termWeights                                                         |
+-----+---------------+--------------------------------------------------------------------+
|0    |[336, 332, 532]|[0.009033212891258758, 0.007667261000964144, 0.0013511317082461137] |
|1    |[58, 19, 12]   |[0.007537349858706455, 0.007315605370934027, 0.007296100365897374]  |
|2    |[6, 734, 782]  |[0.013425615684859974, 0.0013520736783566656, 0.001277673905437223] |
|3    |[667, 8, 740]  |[0.010233026009741185, 0.008338828924965243, 0.0012886921893720548] |
|4    |[44, 1, 56]    |[0.012990841987488821, 0.00875576662804429, 0.0013456833031477216]  |
|5    |[6, 2, 3]      |[0.029841550767097064, 0.01355462742744253, 0.008304519935501157]   |
|6    |[71, 555, 72]  |[0.008022563719229428, 0.007869305917223087, 0.007324476544108817]  |
|7    |[297, 619, 54

### Let's look at out topics
NOTE: More cleaning, filtering, playing around with `CountVectorizer`, and more iterations in `LDA` will result in better Topic Modelling results.

In [18]:
# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())
      + " words):")

topics = ldaModel.describeTopics(50)
topics_rdd = topics.rdd

vocab = mlModel.stages[0].vocabulary

topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()

for idx, topic in enumerate(topics_words):
    print("topic: ", idx)
    print("----------")
    for word in topic:
       print(word)
    print("----------")

Learned topics (as distributions over vocab of 1000 words):
topic:  0
----------
PD
history
extent
education
this review
reconstruction
the blood
signal
RT-PCR
the response
cleavage
healthcare
antigen
UV
intensity
protocol
cannot
Cu
HIV
salt
host
week
the left
efficiency
%
quality
migration
LDL
initiation
access
AIM
synthase
the lack
the diagnosis
the possibility
series
group
theory
MAP
the occurrence
serum
the maximum
MS
INTRODUCTION
December
The expression
plasma
mutant
PA
system
----------
topic:  1
----------
mice
p
<
search
the risk
the other hand
component
MS
Blood
accumulation
impairment
radiation
F
Escherichia
inflammation
preparation
novel
LPS
wall
assay
the presence
CO(2
analysis
the detection
observation
))
agent
S
pretreatment
point
limit
a total
HCV
the efficacy
mg
sensitivity
case
temperature
Further
restriction
sex
coli
understanding
height
mellitus
Hz
a range
pressure
class
diagnosis
----------
topic:  2
----------
+/
December
albumin
maintenance
SC
component
HR
energy
